## Import Dependencies

In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter  
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
from langchain_classic.retrievers import EnsembleRetriever
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains.retrieval import create_retrieval_chain
from typing import List, Dict, Any
import os
from dotenv import load_dotenv

/Users/sumanyadav/Desktop/PROJECT/Lernix/backend/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key is None:
    raise EnvironmentError("OPENAI_API_KEY environment variable not set")
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["TOKENIZERS_PARALLELISM"] = "false"

## Chunking

In [3]:
def chunk_text(text: str, chunk_size: int = 500, overlap: int = 50) -> List[str]:
    splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", " ", ""],
        chunk_size=chunk_size,
        chunk_overlap=overlap,
        length_function=len
    )
    return splitter.split_text(text)

## Convert chunks to Documents

In [4]:
def convert_to_document(chunks: List[str]) -> List[Document]:
    return [Document(page_content=chunk) for chunk in chunks]

## create hybrid retriever

In [5]:
def create_retriever(docs: List[Document]):
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

    dense_vectorstore = FAISS.from_documents(docs, embeddings)
    dense_retriever = dense_vectorstore.as_retriever(search_kwargs={"k": 10})

    sparse_retriever = BM25Retriever.from_documents(docs)
    sparse_retriever.k = 3

    return EnsembleRetriever(
        retrievers=[dense_retriever, sparse_retriever],
        weights=[0.7, 0.3]
    )

## Build RAG Chain

In [6]:
def build_rag_chain(retriever):
    prompt = PromptTemplate.from_template(
        """Summarize the following content clearly and concisely:

        {context}
        """
    )

    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0.4
    )

    document_chain = create_stuff_documents_chain(
        llm=llm,
        prompt=prompt
    )

    return create_retrieval_chain(
        retriever=retriever,
        combine_docs_chain=document_chain
    )

## Summarizer Function

In [7]:
def summarize_text(text: str) -> str:
    chunks = chunk_text(text)
    docs = convert_to_document(chunks)
    retriever = create_retriever(docs)
    rag_chain = build_rag_chain(retriever)
    result = rag_chain.invoke({
        "input": "Summarize the document"
    })

    return result["answer"]


## Testing Result

In [8]:
summarize_text("""
Interpersonal Communication with StrangersInterpersonal communication with strangers refers to the exchange of information, thoughts, andfeelings between people who do not know each other well. This type of communication commonlyoccurs in daily life, such as when speaking with classmates, coworkers, shopkeepers, serviceproviders, or people met for the first time. Effective communication with strangers requires clarity,politeness, and respect. Since there is little or no prior relationship, first impressions are veryimportant. Simple actions like maintaining eye contact, using a friendly tone, and practicing goodlistening skills help create a positive interaction. Clear language and appropriate body posture alsoreduce misunderstandings. Active listening plays a key role in interpersonal communication. Itinvolves paying attention, avoiding interruptions, and responding thoughtfully. When communicatingwith strangers, asking relevant questions and acknowledging their responses shows interest andrespect, helping build trust quickly. Nonverbal communication is equally important. Facialexpressions, gestures, and personal space can strongly influence how a message is received.Being aware of cultural differences is essential, as gestures and communication styles may varyacross cultures. Safety and boundaries should always be maintained while interacting withstrangers. Personal information should not be shared unnecessarily, and communication shouldremain appropriate and professional. Overall, effective interpersonal communication with strangershelps develop social skills, build confidence, and create positive connections in everyday life.
""")

'Interpersonal communication with strangers involves exchanging information and feelings with people who are not well-known. This often occurs in everyday situations, such as with classmates or service providers. Effective communication requires clarity, politeness, and respect, as well as the ability to ask relevant questions and acknowledge responses to build trust. Nonverbal cues, such as facial expressions and gestures, are also crucial, and awareness of cultural differences is important. Maintaining safety and boundaries is essential, as first impressions significantly impact interactions. Key skills include active listening, maintaining eye contact, using a friendly tone, and practicing good body language to foster positive connections.'